# Derivados Financieros 2021 - Clase 5
## Otros modelos de pricing - QuantLib

    3) QuantLib
 

# 3) QuantLib o Introduccion a una Verdadera plataforma de pricing/riesgo

In [3]:
import sys
sys.path.append('..')
from Codigo.utils_plots import *
import QuantLib as ql

### Opciones

### Supongamos una opcion de AAPL con precio strike de 450 expirando el 16 de octubre de 2020. Supongamos que el spot es 440. La vol implicita es sabida que es 31%, y tiene un dividendo de 0.75%. Valuemos la opcion al dia de hoy (10 de Agosto 2020)

#### Definimos los inputs

In [4]:
S = 456.30
K = 460
sigma = 0.3394 # the historical vols for a year
div =  0.075
r = 0.0013

option_type = ql.Option.Call

option_type

1

#### Para las fechas QuantLib es mas preciso. Pide convenciones y calendarios

In [5]:
maturity_date = ql.Date(16, 10, 2020)
calculation_date = ql.Date(6, 8, 2020)

day_count = ql.Actual365Fixed()

ql.Argentina()

calendar = ql.UnitedStates()
ql.Settings.instance().evaluationDate = calculation_date

maturity_date

Date(16,10,2020)

#### Aqui construimos a la opcion europea

In [13]:
payoff = ql.PlainVanillaPayoff(option_type, K)

exercise = ql.EuropeanExercise(maturity_date)

european_option = ql.VanillaOption(payoff, exercise)



#### Aca constuimos el proceso de Black Sholes Merton

##### El objeto spot

In [6]:
spot_obj = ql.QuoteHandle(ql.SimpleQuote(S))

In [7]:
spot_obj.value()

456.3

##### El objeto curva de descuento (flat en este caso, constante)

In [8]:
rate_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                            r, 
                                                    day_count))

##### El objeto curva de dividendos (flat en este caso, constante)

In [9]:
dividend_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                      div, 
                                                      day_count))

##### El objeto volatilidad (flat en este caso, constante)

In [10]:
vol_obj = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, 
                                                                 calendar, 
                                                                 sigma, 
                                                             day_count))

In [63]:
##### El proceso propiamente dicho

In [11]:
bsm_process = ql.BlackScholesMertonProcess(spot_obj, 
                                        dividend_obj, 
                                        rate_obj, 
                                        vol_obj)

## Modelos de precio

###  `AnalyticEuropeanEngine` (Black Scholes)

In [14]:
european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))

bs_price = european_option.NPV()
print("El precio teorico usando el modelo de BS es: ", bs_price)

El precio teorico usando el modelo de BS es:  22.340433141477693


###  `FdBlackScholesVanillaEngine` (Diferencias Finitas)

In [15]:
european_option.setPricingEngine(ql.FdBlackScholesVanillaEngine(bsm_process))

fd_price = european_option.NPV()
print("El precio teorico usando el modelo de Dif. Finitas es: ", fd_price)

El precio teorico usando el modelo de Dif. Finitas es:  22.350547133690334


###  `MCEuropeanEngine` (Montecarlo)

In [16]:
european_option.setPricingEngine(ql.MCEuropeanEngine(bsm_process, 
"PseudoRandom", 
timeSteps=20, requiredSamples=500000))

mc_price = european_option.NPV()
print("El precio teorico usando el modelo de MonteCarlo es: ",  mc_price)

El precio teorico usando el modelo de MonteCarlo es:  22.219725772915616


In [17]:
european_option.errorEstimate()

0.05572959878585315

###  `BinomialVanillaEngine` (Binomial)

In [18]:
european_option.setPricingEngine(ql.BinomialVanillaEngine(bsm_process, "crr", 1000))

bin_price = european_option.NPV()
print("El precio teorico usando el modelo Binomial es: ",  bin_price)

El precio teorico usando el modelo Binomial es:  22.340372556198265


# Comparacion de TODOS* los modelos vistos (caso de juguete)

### Fijo los parametros

In [19]:
S = 100
K = 100
sigma = 0.25 # the historical vols for a year
div =  0.0
r = 0.05

tipo = 'C'

option_type = ql.Option.Call

maturity_date = ql.Date(10, 8, 2021)
calculation_date = ql.Date(10, 8, 2020)
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()
ql.Settings.instance().evaluationDate = calculation_date

T = 1



## Pricers vistos en clase

### Importo los pricers vistos en clase

In [20]:
from Codigo.opcion_europea_bs import opcion_europea_bs
from Codigo.opcion_europea_mc import opcion_europea_mc
from Codigo.opcion_europea_fd import opcion_europea_fd
from Codigo.opcion_europea_bin import opcion_europea_bin

import timeit

### Corro cada uno de los pricers (les tomo el tiempo de ejecucion tambien)

In [21]:
start = timeit.default_timer()
precio_bs = opcion_europea_bs(tipo, S, K, T, r, sigma, div)
end = timeit.default_timer()
tiempo_bs = end - start


start = timeit.default_timer()
precio_bin = opcion_europea_bin(tipo, S, K, T, r, sigma, div, 5000)
end = timeit.default_timer()
tiempo_bin = end - start

start = timeit.default_timer()
precio_mc = opcion_europea_mc(tipo, S, K, T, r, sigma, div, 5000)
end = timeit.default_timer()
tiempo_mc = end - start

start = timeit.default_timer()
precio_fd = opcion_europea_fd(tipo, S, K, T, r, sigma, div)
end = timeit.default_timer()
tiempo_fd = end - start

## Precios QuantLib

### Seteo lo necesario en QuantLib

In [22]:
payoff = ql.PlainVanillaPayoff(option_type, K)
exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)

spot_obj = ql.QuoteHandle(ql.SimpleQuote(S))

rate_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                            r, 
                                                    day_count))

dividend_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                      div, 
                                                      day_count))

vol_obj = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, 
                                                                 calendar, 
                                                                 sigma, 
                                                             day_count))

bsm_process = ql.BlackScholesMertonProcess(spot_obj, 
                                        dividend_obj, 
                                        rate_obj, 
                                        vol_obj)



### Corro cada uno de los pricers (les tomo el tiempo de ejecucion tambien)

In [23]:

start = timeit.default_timer()
european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
bs_price = european_option.NPV()
end = timeit.default_timer()
time_bs = end - start

start = timeit.default_timer()
european_option.setPricingEngine(ql.FdBlackScholesVanillaEngine(bsm_process))
fd_price = european_option.NPV()
end = timeit.default_timer()
time_fd = end - start

start = timeit.default_timer()
european_option.setPricingEngine(ql.MCEuropeanEngine(bsm_process, "PseudoRandom", timeSteps=20, requiredSamples=5000))
mc_price = european_option.NPV()
end = timeit.default_timer()
time_mc = end - start

start = timeit.default_timer()
european_option.setPricingEngine(ql.BinomialVanillaEngine(bsm_process, "crr", 5000))
bin_price = european_option.NPV()
end = timeit.default_timer()
time_bin = end - start



### Imprimo TODO

In [24]:
print("Precio modelo Europea BS visto en clase:", precio_bs)
print("Tiempo modelo Europea BS visto en clase:", tiempo_bs)

print("Precio modelo Europea Binomial visto en clase:", precio_bin)
print("Tiempo modelo Europea Binomial visto en clase:", tiempo_bin)

print("Precio modelo Europea Montecarlo visto en clase:", precio_mc)
print("Tiempo modelo Europea Montecarlo visto en clase:", tiempo_mc)

print("Precio modelo Europea Dif. Finitas visto en clase:", precio_fd)
print("Tiempo modelo Europea Dif. Finitas visto en clase:", tiempo_fd)


print("Precio modelo Europea BS QuantLib", bs_price)
print("Tiempo modelo Europea BS QuantLib:", time_bs)

print("Precio modelo Europea Binomial QuantLib:", bin_price)
print("Tiempo modelo Europea Binomial QuantLib:", time_bin)

print("Precio modelo Europea Montecarlo QuantLib:", mc_price)
print("Tiempo modelo Europea Montecarlo QuantLib:", time_mc)

print("Precio modelo Europea Dif. Finitas QuantLib:", fd_price)
print("Tiempo modelo Europea Dif. Finitas QuantLib:", time_fd)

Precio modelo Europea BS visto en clase: 12.335998930368717
Tiempo modelo Europea BS visto en clase: 0.0008471999317407608
Precio modelo Europea Binomial visto en clase: 12.33550453323051
Tiempo modelo Europea Binomial visto en clase: 40.68674450018443
Precio modelo Europea Montecarlo visto en clase: 12.664345225015463
Tiempo modelo Europea Montecarlo visto en clase: 0.010519500123336911
Precio modelo Europea Dif. Finitas visto en clase: 12.32673769241634
Tiempo modelo Europea Dif. Finitas visto en clase: 0.7286076999735087
Precio modelo Europea BS QuantLib 12.335998930368715
Tiempo modelo Europea BS QuantLib: 0.00033760000951588154
Precio modelo Europea Binomial QuantLib: 12.335493341380698
Tiempo modelo Europea Binomial QuantLib: 0.12747969990596175
Precio modelo Europea Montecarlo QuantLib: 12.351062703379522
Tiempo modelo Europea Montecarlo QuantLib: 0.07314089988358319
Precio modelo Europea Dif. Finitas QuantLib: 12.343995594762795
Tiempo modelo Europea Dif. Finitas QuantLib: 0.00

: 

### Tabla de precios por modelo

| Precio | Clase | QuantLib |
| --- | --- | --- |
| BS | 12.336 | 12.336 |
| Binomial | 12.33355 | 12.33347 |
| Montecarlo | _12.33747_ | _12.38694_ |
| Dif.Finitas | 12.33380 | 12.34399 |

### Tabla de tiempos de ejecucion por modelo**

| Tiempo | Clase | QuantLib |
| --- | --- | --- |
| BS | **0.00033** | **0.00023** |
| Binomial | 17.38176 | **0.06012** |
| Montecarlo | **0.72694** | 3.03739 |
| Dif.Finitas | 0.046359 | **0.001721** |

### Opciones Americanas


In [78]:
S = 100
K = 100
sigma = 0.25 # the historical vols for a year
div =  0.0
r = 0.05
tipo = 'C'

option_type = ql.Option.Put

maturity_date = ql.Date(10, 8, 2021)
calculation_date = ql.Date(10, 8, 2020)
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()
ql.Settings.instance().evaluationDate = calculation_date

T = 1


In [79]:
payoff = ql.PlainVanillaPayoff(option_type, K)

am_exercise = ql.AmericanExercise(calculation_date, maturity_date)
american_option = ql.VanillaOption(payoff, am_exercise)

In [80]:
spot_obj = ql.QuoteHandle(ql.SimpleQuote(S))

rate_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                            r, 
                                                    day_count))

dividend_obj = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, 
                                                      div, 
                                                      day_count))

vol_obj = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, 
                                                                 calendar, 
                                                                 sigma, 
                                                             day_count))

bsm_process = ql.BlackScholesMertonProcess(spot_obj, 
                                        dividend_obj, 
                                        rate_obj, 
                                        vol_obj)

In [81]:
# No hay Black Scholes
american_option.setPricingEngine(ql.BinomialVanillaEngine(bsm_process, "crr", 1000))
bin_price_ame = american_option.NPV()

american_option.setPricingEngine(ql.FdBlackScholesVanillaEngine(bsm_process))
fd_price_ame = american_option.NPV()

american_option.setPricingEngine(ql.MCAmericanEngine(bsm_process, "PseudoRandom", timeSteps=20, requiredSamples=500000))
mc_price_ame = american_option.NPV()



In [82]:
american_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
bin_price_ame = american_option.NPV()

RuntimeError: not an European option

In [83]:
from Codigo.opcion_americana_bin import opcion_americana_bin
from Codigo.opcion_americana_fd import opcion_americana_fd

In [84]:
tipo = "P"
precio_bin_ame = opcion_americana_bin(tipo, S, K, T, r, sigma, div, 1000)
precio_fd_ame = opcion_americana_fd(tipo, S, K, T, r, sigma, div)


In [85]:
print("Precio modelo Amercana Binomial visto en clase:", precio_bin_ame)
print("Precio modelo Amercana Dif. Finitas visto en clase:", precio_fd_ame)

print("Precio modelo Amercana Binomial QuantLib:", bin_price_ame)
print("Precio modelo Amercana Montecarlo QuantLib:", mc_price_ame)
print("Precio modelo Amercana Dif. FinitasQuantLib:", fd_price_ame)

Precio modelo Amercana Binomial visto en clase: 7.973040110898395
Precio modelo Amercana Dif. Finitas visto en clase: 8.076404881004764
Precio modelo Amercana Binomial QuantLib: 7.973463892545318
Precio modelo Amercana Montecarlo QuantLib: 7.839081658384568
Precio modelo Amercana Dif. FinitasQuantLib: 7.966283110785785
